In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Fast Food Analysis Project Spreadsheet.xlsx")

In [3]:
df.head()

,Unnamed: 0,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g),Item Type
0,NaN,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2,Breakfast
1,NaN,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2,Breakfast
2,NaN,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2,Entrée
3,NaN,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side
4,NaN,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.head()

,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g),Item Type
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2,Breakfast
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2,Breakfast
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2,Entrée
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée


In [6]:
df.shape

(49, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Restaurant        49 non-null     object 
 1   Item Name         49 non-null     object 
 2   Calories          49 non-null     int64  
 3   Protein (g)       49 non-null     int64  
 4   Total Fats (g)    49 non-null     int64  
 5   Sodium (mg)       49 non-null     int64  
 6   Total Sugars (g)  49 non-null     int64  
 7   Cholestrol (mg)   49 non-null     int64  
 8   Price ($)         49 non-null     float64
 9   Carbs (g)         49 non-null     int64  
 10  Fiber (g)         49 non-null     int64  
 11  Item Type         49 non-null     object 
dtypes: float64(1), int64(8), object(3)
memory usage: 4.7+ KB


In [8]:
print(df.columns)

Index(['Restaurant', 'Item Name', 'Calories ', 'Protein (g)', 'Total Fats (g)',
       'Sodium (mg)', 'Total Sugars (g)', 'Cholestrol (mg)', 'Price ($)',
       'Carbs (g)', 'Fiber (g)', 'Item Type'],
      dtype='object')


In [9]:
df = df.rename(columns={'Restaurant': 'restaurant_name', 'Item Name': 'item_name', 'Calories ': 'calories', 'Protein (g)': 'protein_g',
 'Total Fats (g)': 'fats_g', 'Sodium (mg)': 'sodium_mg', 'Total Sugars (g)': 'sugars_g',
 'Cholestrol (mg)': 'cholestrol_mg', 'Price ($)': 'price', 'Carbs (g)': 'carbs_g', 
 'Fiber (g)': 'fiber_g', 'Item Type': 'item_type'})
df.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2,Breakfast
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2,Breakfast
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2,Entrée
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée


In [10]:
df.isnull().sum()


restaurant_name    0
item_name          0
calories           0
protein_g          0
fats_g             0
sodium_mg          0
sugars_g           0
cholestrol_mg      0
price              0
carbs_g            0
fiber_g            0
item_type          0
dtype: int64

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
## starting tier one analysis: basic averages, ranks of each restaurant and item with the highest calories, protein etc. 
df.describe()

,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,444.693878,20.204082,22.979592,1018.469388,8.959184,68.265306,6.604286,40.306122,2.938776
std,226.265902,14.850617,14.138437,733.132693,16.724432,64.835939,2.719396,21.897683,3.044176
min,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.820000,10.000000,0.000000
25%,270.000000,7.000000,14.000000,380.000000,2.000000,20.000000,4.550000,26.000000,1.000000
50%,430.000000,18.000000,21.000000,1010.000000,5.000000,60.000000,6.330000,42.000000,2.000000
75%,570.000000,29.000000,31.000000,1420.000000,10.000000,90.000000,7.490000,46.000000,4.000000
max,1270.000000,61.000000,78.000000,3070.000000,113.000000,315.000000,13.980000,129.000000,14.000000


In [13]:
## gonna use a loop to find top and bottom 10 of each metric (protein, fiber etc)
metrics = ['calories', 'protein_g', 'fats_g', 'sodium_mg', 'cholestrol_mg', 'price', 'carbs_g', 'fiber_g']

for metric in metrics:
    print(f"Top 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).head(10))

    print(f"Bottom 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).tail(10))

Top 10 Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
40,Zaxbys,Chicken Bacon Ranch Loaded Fries,1270,61,78,3070,2,145,12.39,88,8,Side
43,Zaxbys,The Fried Cobb Zalad,780,55,45,2130,5,315,10.55,43,4,Entrée
32,Taco Bell,Nachos BellGrande (Beef),730,17,39,1180,4,20,8.16,80,13,Entrée
46,Zaxbys,Birthday Cake Milkshake,730,13,19,600,113,60,3.30,129,0,Desert
34,Taco Bell,Grilled Cheese Burrito,710,25,39,1510,4,75,6.82,64,6,Entrée
39,Zaxbys,Fried White Cheddar Bites,700,27,46,1550,2,75,7.11,46,3,Side
42,Zaxbys,Kickin Chicken Sandwich,690,34,39,2760,5,65,10.02,51,2,Entrée
17,Chick-fil-A,Smokehouse BBQ Bacon Sandwich w/ Reg Filet,670,41,31,1930,12,110,10.45,57,3,Entrée
45,Zaxbys,The Fried House Zalad,660,44,37,1920,4,90,13.98,42,4,Entrée
16,Chick-fil-A,Cool Wrap,660,43,45,1420,5,110,11.25,32,14,Entrée


Bottom 10 Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side
27,Starbucks,Tall Caffé Mocha,200,7,9,75,18,30,6.15,22,2,Drink
33,Taco Bell,Nacho Cheese Doritos Locos Tacos Supreme,190,8,11,380,1,30,4.62,15,3,Entrée
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée
36,Taco Bell,Cinnamon Twists,170,1,6,115,10,0,1.82,27,1,Desert
26,Starbucks,Birthday Cake Pop,160,2,8,95,16,10,3.25,21,0,Desert
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0,Drink
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0,Drink
23,Starbucks,Tall Strawberry Açaí Lemonade Refresher,80,0,0,10,16,0,5.95,18,1,Drink
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0,Drink


Top 10 Protein_G


...

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
23,Starbucks,Tall Strawberry Açaí Lemonade Refresher,80,0,0,10,16,0,5.95,18,1,Drink
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0,Drink
11,Chick-fil-A,Small Mac & Cheese,270,12,17,710,2,40,4.55,17,2,Side
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0,Entrée
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0,Drink
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0,Entrée
33,Taco Bell,Nacho Cheese Doritos Locos Tacos Supreme,190,8,11,380,1,30,4.62,15,3,Entrée
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0,Drink
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0,Entrée
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée


Top 10 Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
16,Chick-fil-A,Cool Wrap,660,43,45,1420,5,110,11.25,32,14,Entrée
32,Taco Bell,Nachos BellGrande (Beef),730,17,39,1180,4,20,8.16,80,13,Entrée
37,Taco Bell,Mexican Pizza,530,19,28,1010,3,40,7.31,50,8,Entrée
40,Zaxbys,Chicken Bacon Ranch Loaded Fries,1270,61,78,3070,2,145,12.39,88,8,Side
31,Taco Bell,Beefy 5-Layer Burrito,490,17,18,1290,4,30,5.11,65,7,Entrée
30,Taco Bell,Crunchwrap Supreme,530,15,20,1220,6,20,7.31,73,6,Entrée
34,Taco Bell,Grilled Cheese Burrito,710,25,39,1510,4,75,6.82,64,6,Entrée
13,Chick-fil-A,Medium Waffle Potato Fries,420,5,24,240,1,0,3.59,45,5,Side
44,Zaxbys,Regular Crinkle Fries,330,5,14,580,0,0,3.81,47,5,Side
38,Taco Bell,Cheesy Gordita Crunch,490,20,28,850,4,55,6.33,42,5,Entrée


Bottom 10 Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type
20,Starbucks,"Sausage, Cheddar, & Egg Sandwich",480,18,29,890,2,165,5.95,34,1,Breakfast
46,Zaxbys,Birthday Cake Milkshake,730,13,19,600,113,60,3.30,129,0,Desert
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0,Drink
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0,Drink
26,Starbucks,Birthday Cake Pop,160,2,8,95,16,10,3.25,21,0,Desert
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0,Drink
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0,Entrée
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0,Entrée
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0,Entrée


In [14]:
#now gonna do a loop for the averages (gonna use medians, chose due to discovery of outliers) based on restaurants
for metric in metrics:
    print(f"Restaurants Ranked by Median {metric.title()} (High to Low)")

    ranked = df.groupby('restaurant_name')[metric].median().sort_values(ascending=False)
    display(ranked)

Restaurants Ranked by Median Calories (High to Low)


restaurant_name
Zaxbys         675.0
Taco Bell      510.0
Chick-fil-A    455.0
McDonald's     400.0
Starbucks      180.0
Name: calories, dtype: float64

Restaurants Ranked by Median Protein_G (High to Low)


restaurant_name
Zaxbys         32.5
Chick-fil-A    30.5
McDonald's     17.0
Taco Bell      17.0
Starbucks       5.0
Name: protein_g, dtype: float64

Restaurants Ranked by Median Fats_G (High to Low)


...

restaurant_name
Zaxbys         9.360
Chick-fil-A    7.740
Taco Bell      6.575
Starbucks      6.050
McDonald's     4.290
Name: price, dtype: float64

Restaurants Ranked by Median Carbs_G (High to Low)


restaurant_name
Zaxbys         46.5
Taco Bell      46.0
Chick-fil-A    42.0
McDonald's     38.0
Starbucks      21.5
Name: carbs_g, dtype: float64

Restaurants Ranked by Median Fiber_G (High to Low)


restaurant_name
Taco Bell      5.5
Zaxbys         3.0
Chick-fil-A    2.0
McDonald's     2.0
Starbucks      1.0
Name: fiber_g, dtype: float64

In [15]:
#do the same as above but by item type now
for metric in metrics:
    print(f"Item Types Ranked by Median {metric.title()} (High to Low)")

    ranked = df.groupby('item_type')[metric].median().sort_values(ascending=False)
    display(ranked)

Item Types Ranked by Median Calories (High to Low)


item_type
Entrée       525.0
Breakfast    430.0
Side         375.0
Desert       210.0
Drink        125.0
Name: calories, dtype: float64

Item Types Ranked by Median Protein_G (High to Low)


item_type
Entrée       29.0
Breakfast    18.0
Side          7.0
Drink         4.5
Desert        3.5
Name: protein_g, dtype: float64

Item Types Ranked by Median Fats_G (High to Low)


...

item_type
Entrée       7.310
Drink        6.150
Breakfast    5.950
Side         3.880
Desert       3.275
Name: price, dtype: float64

Item Types Ranked by Median Carbs_G (High to Low)


item_type
Side         45.5
Entrée       42.5
Breakfast    35.0
Desert       26.5
Drink        18.0
Name: carbs_g, dtype: float64

Item Types Ranked by Median Fiber_G (High to Low)


item_type
Side         3.5
Entrée       3.0
Breakfast    2.0
Desert       0.5
Drink        0.5
Name: fiber_g, dtype: float64

In [16]:
## starting tier two analysis
## start of the clustering technique 
df.describe()


,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,444.693878,20.204082,22.979592,1018.469388,8.959184,68.265306,6.604286,40.306122,2.938776
std,226.265902,14.850617,14.138437,733.132693,16.724432,64.835939,2.719396,21.897683,3.044176
min,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.820000,10.000000,0.000000
25%,270.000000,7.000000,14.000000,380.000000,2.000000,20.000000,4.550000,26.000000,1.000000
50%,430.000000,18.000000,21.000000,1010.000000,5.000000,60.000000,6.330000,42.000000,2.000000
75%,570.000000,29.000000,31.000000,1420.000000,10.000000,90.000000,7.490000,46.000000,4.000000
max,1270.000000,61.000000,78.000000,3070.000000,113.000000,315.000000,13.980000,129.000000,14.000000


In [17]:
## making definitions of clusters + finding how many there are; will in next iteration of project separate by item type! ##
cluster_rules = {
    'high_protein_low_fat': (df['protein_g'] >= 20) & (df['fats_g'] <= 23),
    'low_carb_high_fat': (df['carbs_g'] <= 40) & (df['fats_g'] >= 23),
    'high_fiber_low_sodium': (df['fiber_g'] >= 3) & (df['sodium_mg'] <= 1015),
    'high_protein_low_price': (df['protein_g'] >= 20) & (df['price'] <= 6.60), 
    'low_calorie_high_fiber': (df['calories'] <= 440) & (df['fiber_g'] >= 3)
}

for name, rule in cluster_rules.items():
    df[f"cluster_{name}"] = rule

In [18]:
for name in cluster_rules:
    print(f"{name}: {df[f'cluster_{name}'].sum()} items")

high_protein_low_fat: 7 items
low_carb_high_fat: 3 items
high_fiber_low_sodium: 7 items
high_protein_low_price: 3 items
low_calorie_high_fiber: 5 items


In [19]:
cluster_name_map = {
    'high_protein_low_fat': 'High Protein + Low Fat', 
    'low_carb_high_fat': 'Low Carb + High Fat', 
    'high_fiber_low_sodium': 'High Fiber + Low Sodium', 
    'high_protein_low_price': 'High Protein + Low Price', 
    'low_calorie_high_fiber': 'Low Calorie + High Fiber'
}

df['manual_cluster'] = 'None'

for name in cluster_rules:
    clean_label = cluster_name_map[name]
    df.loc[df[f'cluster_{name}'], 'manual_cluster'] = clean_label

In [20]:
df.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,cluster_high_protein_low_fat,cluster_low_carb_high_fat,cluster_high_fiber_low_sodium,cluster_high_protein_low_price,cluster_low_calorie_high_fiber,manual_cluster
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2,Breakfast,False,False,False,False,False,None
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2,Breakfast,False,False,False,False,False,None
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2,Entrée,False,False,False,False,False,None
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side,False,False,True,False,True,Low Calorie + High Fiber
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée,False,False,False,False,False,None


In [21]:
df[df['manual_cluster'] != 'None'].sort_values('manual_cluster')

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,cluster_high_protein_low_fat,cluster_low_carb_high_fat,cluster_high_fiber_low_sodium,cluster_high_protein_low_price,cluster_low_calorie_high_fiber,manual_cluster
37,Taco Bell,Mexican Pizza,530,19,28,1010,3,40,7.31,50,8,Entrée,False,False,True,False,False,High Fiber + Low Sodium
15,Chick-fil-A,Chicken Sandwich,420,29,18,1460,6,70,7.09,41,1,Entrée,True,False,False,False,False,High Protein + Low Fat
18,Chick-fil-A,Deluxe Sandwich,490,32,22,1700,7,85,7.99,43,1,Entrée,True,False,False,False,False,High Protein + Low Fat
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0,Entrée,True,False,False,False,False,High Protein + Low Fat
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0,Entrée,True,False,False,False,False,High Protein + Low Fat
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0,Entrée,True,False,False,False,False,High Protein + Low Fat
10,Chick-fil-A,Grilled Chicken Club Sandwich,520,37,22,1055,12,105,11.55,45,3,Entrée,True,False,False,False,False,High Protein + Low Fat
8,McDonald's,Big Mac,580,25,34,1060,7,85,6.59,45,3,Entrée,False,False,False,True,False,High Protein + Low Price
5,McDonald's,McDouble,390,22,20,920,6,70,4.29,32,2,Entrée,True,False,False,True,False,High Protein + Low Price
38,Taco Bell,Cheesy Gordita Crunch,490,20,28,850,4,55,6.33,42,5,Entrée,False,False,True,True,False,High Protein + Low Price


In [22]:
## for easier viewing of the items and their clusters ##
cluster_cols = [col for col in df.columns if col.startswith('cluster_')]

df_display = df.drop(columns=cluster_cols)

In [23]:
df_display.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2,Breakfast,None
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2,Breakfast,None
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2,Entrée,None
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side,Low Calorie + High Fiber
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0,Entrée,None


In [24]:
df[df['manual_cluster'] == 'High Fiber + Low Sodium']

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,cluster_high_protein_low_fat,cluster_low_carb_high_fat,cluster_high_fiber_low_sodium,cluster_high_protein_low_price,cluster_low_calorie_high_fiber,manual_cluster
37,Taco Bell,Mexican Pizza,530,19,28,1010,3,40,7.31,50,8,Entrée,False,False,True,False,False,High Fiber + Low Sodium


In [25]:
#now gonna do a loop to rank items in each cluster by metrics
for metric in metrics:
    for cluster in df['manual_cluster'].unique():
        if cluster != 'None':
            cluster_df = df_display[df['manual_cluster'] == cluster]
            sorted_df = cluster_df.sort_values(by=metric, ascending=False)

            print(f"Ranked Items in Cluster: {cluster} by {metric.title()}")
            display(sorted_df)

Ranked Items in Cluster: Low Calorie + High Fiber by Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
13,Chick-fil-A,Medium Waffle Potato Fries,420,5,24,240,1,0,3.59,45,5,Side,Low Calorie + High Fiber
35,Taco Bell,Nacho Fries,330,4,19,770,1,0,3.16,36,4,Side,Low Calorie + High Fiber
44,Zaxbys,Regular Crinkle Fries,330,5,14,580,0,0,3.81,47,5,Side,Low Calorie + High Fiber
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3,Side,Low Calorie + High Fiber
33,Taco Bell,Nacho Cheese Doritos Locos Tacos Supreme,190,8,11,380,1,30,4.62,15,3,Entrée,Low Calorie + High Fiber


Ranked Items in Cluster: High Protein + Low Price by Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
8,McDonald's,Big Mac,580,25,34,1060,7,85,6.59,45,3,Entrée,High Protein + Low Price
38,Taco Bell,Cheesy Gordita Crunch,490,20,28,850,4,55,6.33,42,5,Entrée,High Protein + Low Price
5,McDonald's,McDouble,390,22,20,920,6,70,4.29,32,2,Entrée,High Protein + Low Price


Ranked Items in Cluster: Low Carb + High Fat by Calories


...

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
16,Chick-fil-A,Cool Wrap,660,43,45,1420,5,110,11.25,32,14,Entrée,Low Carb + High Fat
7,McDonald's,Sausage McMuffin,400,14,26,760,2,55,3.19,29,2,Breakfast,Low Carb + High Fat
20,Starbucks,"Sausage, Cheddar, & Egg Sandwich",480,18,29,890,2,165,5.95,34,1,Breakfast,Low Carb + High Fat


Ranked Items in Cluster: High Protein + Low Fat by Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
10,Chick-fil-A,Grilled Chicken Club Sandwich,520,37,22,1055,12,105,11.55,45,3,Entrée,High Protein + Low Fat
15,Chick-fil-A,Chicken Sandwich,420,29,18,1460,6,70,7.09,41,1,Entrée,High Protein + Low Fat
18,Chick-fil-A,Deluxe Sandwich,490,32,22,1700,7,85,7.99,43,1,Entrée,High Protein + Low Fat
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0,Entrée,High Protein + Low Fat
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0,Entrée,High Protein + Low Fat
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0,Entrée,High Protein + Low Fat


Ranked Items in Cluster: High Fiber + Low Sodium by Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g,item_type,manual_cluster
37,Taco Bell,Mexican Pizza,530,19,28,1010,3,40,7.31,50,8,Entrée,High Fiber + Low Sodium


In [27]:
## from this code, we can see if items are matching for multiple clusters
df[[col for col in df.columns if col.startswith('cluster_')]].sum(axis=1).value_counts()

0    31
1    11
2     7
Name: count, dtype: int64